## Quantitative Methods 2:  Data Science and Visualisation
## Workshop 9: Using geopandas to create more complex maps

We've successfully plotted spatial data using the core pandas and matplotlib libraries. In this session, we will use *geopandas* - a library which is designed explicitly to work with apatial data; it does projection and allows us to plot geometries and create *choropleth* maps.

### Aims
- Create and view a geopandas dataframe
- Understand the significance of the *geometry* column
- The basics of projection
- Create a choropleth map, based on data
- Join dataframes to match data to geographies
- Plot point data (extension)
- Plot point data with a basemap (extension)

## Install packages
Firstly, we will need to install Geopandas and pysal so that we can use it within this workshop.  This will generate alot of text and might take a minute to complete. This is how we install packages within Azure Notebooks.

In [ ]:
#Install newest branch
!pip install pysal

#Install the geopandas module
!pip install geopandas

## Downloading the Data
Let's grab the data we will need this week from our course website and save it into our data folder. If you've not already created a data folder then do so using the following command. 

Don't worry if it generates an error, that means you've already got a data folder.

In [ ]:
!mkdir data

In [ ]:
!mkdir data/wk9
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk9/london_wards.shp -o ./data/wk9/london_wards.shp
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk9/london_wards.cpg -o ./data/wk9/london_wards.cpg
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk9/london_wards.dbf -o ./data/wk9/london_wards.dbf
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk9/london_wards.prj -o ./data/wk9/london_wards.prj
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk9/london_wards.shx -o ./data/wk9/london_wards.shx
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk9/2011persons.csv -o ./data/wk9/2011persons.csv
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk9/borough_density.csv -o ./data/wk9/borough_density.csv
!curl https://s3.eu-west-2.amazonaws.com/qm2/wk9/tweet_data.csv -o ./data/wk9/tweet_data.csv

## Setup our enviroment

In [ ]:
!pip install descartes
!pip install mapclassify

In [ ]:
from pysal import *
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pylab
import descartes
import mapclassify

%matplotlib inline

plt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = (20., 16.)

## The shape of things to come
Geopandas works with *shapefiles* - an industry standard in spatial analysis and GIS created by ESRI, who make a lot of GIS products. Shapefiles actually involves a number of files (*filename.shp*, *filename.shx*, *filename.dbf*, *filename.proj* and *filename.cpg*) which encode the polygons, data, projection and various other useful bits of data. Suffice it to say that we generally need all of these bits, with the same filename (but different extension), in the same folder. Generally, if you download shapefiles, it should have all of the requisite bits.

We'll load them into a *GeoDataFrame* object, which, as we will see, looks a lot like a pandas dataframe - but has a special column, *geometry*, which is what is used whenever we invoke the "plot" command.

Let's load the data - the geometries of London wards:

In [ ]:
data_path = "./data/wk9/london_wards.shp"

# londonWards = gp.GeoDataFrame.from_file(data_path)

londonWards = gp.read_file(data_path)

londonWards.head()

## Projection

We could spend a long time working through the merits of different projection methods; many of these are available in geopandas, through the PySAL library - more documentation is available there if you want to work with different projections.

More fundamentally, a *Co-ordinate Reference System* (CRS) refers to how we identify points on the globe; the world isn't a perfect sphere so dealing with that is the job of the CRS. The WGS84 datum (https://en.wikipedia.org/wiki/World_Geodetic_System) which is sometimes called EPSG 4326 (epsg:4326). 

We can find out the CRS of the geometry:

In [ ]:
londonWards.crs

If we look at the geometry column, we see that they are latlon co-ordinates:

In [ ]:
londonWards['geometry'].head()

This is all well and good, but we need to project the latlon data onto a plane to plot. We'll create objects to keep track of these CRS frames. *original_crs* is the latlon co-ordinates; *target_crs* is a mercator ('merc') projection; and the third line, *to_crs* projects the geometry so we can plot it.

**Extension**: You can find more detailed documentation about projection types and parameters at the PROJ.4 site which underpins pyproj/geopandas, and here: http://www.remotesensing.org/geotiff/proj_list/; in this case, it won't make much difference which projection you use, because of the small scale.

In [ ]:
londonWards.crs = 'epsg:4326'
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'merc'}
projected_londonWards = londonWards.to_crs(crs=target_crs)

The 'geometry' column isn't in a lon, lat CRS any longer, but physical distances from some spatial point:

In [ ]:
projected_londonWards['geometry'].head()

Plotting this is now a simple matter:

In [ ]:
projected_londonWards.plot()

Our colours look a bit plain and boring.  Let's upgrade our plot to something more informative.

In [ ]:
projected_londonWards.plot(column='UNIT_ID', cmap='rainbow', scheme='quantiles')

Have a play around with mapping differnt coloums using different colours and different colouring schemes

`Scheme must be in the set: ['quantiles', 'fisher_jenks', 'equal_interval']`

If you want to play with the colour schemes for the map then take a look here.
https://matplotlib.org/examples/color/colormaps_reference.html

`Possible values are: Accent, Accent_r, Blues, Blues_r, BrBG, BrBG_r, BuGn, BuGn_r, BuPu, BuPu_r, CMRmap, CMRmap_r, Dark2, Dark2_r, GnBu, GnBu_r, Greens, Greens_r, Greys, Greys_r, OrRd, OrRd_r, Oranges, Oranges_r, PRGn, PRGn_r, Paired, Paired_r, Pastel1, Pastel1_r, Pastel2, Pastel2_r, PiYG, PiYG_r, PuBu, PuBuGn, PuBuGn_r, PuBu_r, PuOr, PuOr_r, PuRd, PuRd_r, Purples, Purples_r, RdBu, RdBu_r, RdGy, RdGy_r, RdPu, RdPu_r, RdYlBu, RdYlBu_r, RdYlGn, RdYlGn_r, Reds, Reds_r, Set1, Set1_r, Set2, Set2_r, Set3, Set3_r, Spectral, Spectral_r, Wistia, Wistia_r, YlGn, YlGnBu, YlGnBu_r, YlGn_r, YlOrBr, YlOrBr_r, YlOrRd, YlOrRd_r, afmhot, afmhot_r, autumn, autumn_r, binary, binary_r, bone, bone_r, brg, brg_r, bwr, bwr_r, cool, cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, flag, flag_r, gist_earth, gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, gist_ncar_r, gist_rainbow, gist_rainbow_r, gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, gnuplot, gnuplot2, gnuplot2_r, gnuplot_r, gray, gray_r, hot, hot_r, hsv, hsv_r, inferno, inferno_r, jet, jet_r, magma, magma_r, nipy_spectral, nipy_spectral_r, ocean, ocean_r, pink, pink_r, plasma, plasma_r, prism, prism_r, rainbow, rainbow_r, seismic, seismic_r, spectral, spectral_r, spring, spring_r, summer, summer_r, terrain, terrain_r, viridis, viridis_r, winter, winter_r`

We can also control the alpha of the map as well if we want to.

In [ ]:
projected_londonWards.plot(column='UNIT_ID', cmap='rainbow', scheme='quantiles', alpha = 0.5)

## Warding off evil 

The electoral wards of London represent voting areas for local councils; let's now compare their populations to see where the population densities are in the city. 

Population data is available from the ONS - they link the population of different ages to a 'Ward Code', which is a standard identified of that ward: http://www.ons.gov.uk/ons/publications/re-reference-tables.html?edition=tcm%3A77-301951

Let's read it into a standard Pandas dataframe:

In [ ]:
data_path = "./data/wk9/2011persons.csv"

persons = pd.read_csv(data_path, encoding = 'latin1')
persons.head()

We need to make sure the data is in the right format - to remove commas and dashes, as we have done before. I'm just going to clean up the "All Ages" column for this example, but you could apply this to any (or all) numerical columns.

In [ ]:
persons.replace(',', '', regex=True, inplace=True)
persons['All Ages'] = persons['All Ages'].replace('-', 'NaN', regex=True).astype('float')
persons['All Ages'].head()

## Exercise 
Using the Ward Code, **merge** the *persons* geodataframe with the *londonWards* dataframe to create a new geodataframe called "geopeople". You can do do this in pretty much exactly the same way you've done for merging 'vanilla' dataframes - refer to the examples from earlier in the term if your syntax is rusty. 

**One important caveat**: to plot the map, you need to be invoking *.plot()* on a **geo**dataframe - so you need to merge **on** the geodataframe, with the dataframe as the argument. You can check this with the *type(geopeople)* command:

In [ ]:
type(geopeople)

Compare to merging **on** persons, which will give you a "pandas.core.frame.DataFrame" object - *not* a geodataframe.

Let's now calculate the population *density* using the 'HECTARES' column:

In [ ]:
geopeople['density']=geopeople['All Ages']/geopeople['HECTARES']
geopeople.head()

NameError: ignored

## Portable data

At this point, we'll save the data into a .csv so it's easy to use in other software. This will be useful.

In [ ]:
data_path = "./data/wk9/borough_density.csv"

geopeople.to_csv(data_path)

And let's project this into a new geodataframe:

In [ ]:
original_crs = geopeople.crs
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'merc'}
projected_geopeople = geopeople.to_crs(crs=target_crs)

In [ ]:
projected_geopeople.plot(column='All Ages')
plt.title('2011 population by Ward')
plt.savefig('./data/wk9/Wards.png')

## Exercise
Create a choropleth map of the population of each ward using *projected_geopeople.plot()* and save it as an image file. Use the optional arguments: 

1. *column*, To specific the 'All Ages' column in the dataset to colour the map based on population, and
2. *colormap*, using the 'Blues' colormap to specify a white-blue colour range

(It looks as if there aren't population estimates for the smaller City of London Wards, so we're missing them - if we wanted to show them, we would need a different dataset)

## Between the Bars

We have a nice map, but no colour scale to explain what the colours mean; unfortunately, this is a place where geopandas falls down and we have to do something intensely complex. The code you'll see below was created by the inimitable Stephan Hugel to fill in the gaps (more for the superkeen here: sensitivecities.com/so-youd-like-to-make-a-map-using-python-EN.html#.VlSfMcq2-K7)


In [ ]:
import numpy as np
import matplotlib

In [ ]:

# Convenience functions for working with colour ramps and bars
def colorbar_index(ncolors, cmap, labels=None, **kwargs):
    """
    This is a convenience function to stop you making off-by-one errors
    Takes a standard colour ramp, and discretizes it,
    then draws a colour bar with correctly aligned labels
    """
    cmap = cmap_discretize(cmap, ncolors)
    mappable = plt.cm.ScalarMappable(cmap=cmap)
    mappable.set_array([])
    mappable.set_clim(-0.5, ncolors+0.5)
    colorbar = matplotlib.pyplot.colorbar(mappable, **kwargs)
    colorbar.set_ticks(np.linspace(0, ncolors, ncolors))
    colorbar.set_ticklabels(range(ncolors))
    if labels:
        colorbar.set_ticklabels(labels)
    return colorbar

def cmap_discretize(cmap, N):
    """
    Return a discrete colormap from the continuous colormap cmap.

        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.

    Example
        x = resize(arange(100), (5,100))
        djet = cmap_discretize(cm.jet, 5)
        imshow(x, cmap=djet)

    """
    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0., 0., 0., 0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N + 1)
    cdict = {}
    for ki, key in enumerate(('red', 'green', 'blue')):
        cdict[key] = [(indices[i], colors_rgba[i - 1, ki], colors_rgba[i, ki]) for i in range(N+1)]
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d" % N, cdict, 1024)

In [ ]:
from pysal.viz.mapclassify import Quantiles

## Them's the breaks
There are various ways to split the data, the default of which is quantiles - to order the data and then split the N data values we have evenly into q chunks, each of which has N/q data points. Consider the median - this splits the dataset into two chunks of size N/2; quartiles split the data into four chunks of size N/4; *quantiles* split the data into q chunks with N/q points. We will specify 5 chunks using the k=5 argument.

(The documentation is here, if you're interested in finding out more: https://github.com/pysal/pysal/blob/master/pysal/contrib/viz/mapping.py)

In [ ]:
breaks = Quantiles(geopeople['density'].values, k=5)
print(breaks)

In [ ]:
print(breaks.bins)

We'll create a set of labels; again, this is a bit beyond what we've done before so don't worry if it appears a bit mysterious.

In [ ]:
bar_labels = ['<=%i'% b for b in breaks.bins]
print(bar_labels)

In [ ]:
projected_geopeople.plot(column='density', cmap='Greens', scheme='quantiles', k=5)
plt.title('2011 population by Ward')

cmap = plt.get_cmap('Greens')
colorbar_index(ncolors=5, cmap=cmap, shrink=0.5, labels=bar_labels)

## Working with Point Data and Choropleths
Let's now go back to our twitter data and plot it on the same axes...

In [ ]:
data_path = "./data/wk9/tweet_data.csv"

tweets = pd.read_csv(data_path, parse_dates = [1], infer_datetime_format = True, encoding = 'latin1')
tweets.head()

We will convert these points to a geometry object - previously, the geometry column stored data associated with the polygons for the wards - we'll build geometries representing points and then convert the dataframe to a geodataframe. Again, the code below is a bit opaque - it's a recipe for creating these geometries.

In [ ]:
from shapely.geometry import Point
tweets['geometry'] = tweets.apply(lambda x: Point(x['Lon'], x['Lat']), axis=1)

## Project points into the same basis
In the next steps, we will 1) convert the dataframe to a geodataframe. 2) sets the CRS (co-ordinate reference system, remember) to the original CRS that we got from the wards data, and 3) projects into the same projection (using mercator, as before)

In [ ]:
geotweets = gp.GeoDataFrame(tweets)
geotweets.crs = original_crs
geotweets.to_crs(crs=target_crs, inplace=True)

In [ ]:
geotweets.head()

In [ ]:
geotweets.plot()

In [ ]:
ax = projected_geopeople.plot(column='density', cmap='Greens', scheme='quantiles', k=5)
plt.title('2011 population by Ward')

cmap = plt.get_cmap('Greens')
colorbar_index(ncolors=5, cmap=cmap, shrink=0.5, labels=bar_labels)

geotweets.plot(ax=ax)

## Extension:
We can make these points a bit bigger by using a 'buffer' - this creates a geometry of circles rather than points:

In [ ]:
geotweets['points'] = geotweets['geometry']
geotweets['geometry'] = gp.GeoSeries(tweets['points']).buffer(200)
geotweets.head()

And we'll add in a dummy variable we can use to make them all the same colour. Again, a bit clunky but means when we put them on a colormap (the object which converts a value to a color), it will give the same colour for each circle.

In [ ]:
geotweets['dummy']=1

Now we can plot this in the same projection, putting geopeople (which has the borough boundaries) on the same axes as the point twitter data.

In [ ]:
ax = projected_geopeople.plot(column='density', cmap='Greens', scheme='quantiles', k=5)
plt.title('2011 population by Ward')

cmap = plt.get_cmap('Greens')
colorbar_index(ncolors=5, cmap=cmap, shrink=0.5, labels=bar_labels)

geotweets.plot(column='dummy', cmap='RdBu', ax=ax)

This has been a bit fiddly, but we have a tweet map with a choropleth of underlying residential population densities. You could look at evening-time tweets only and then we're starting to get somewhere. But this is just an example, so you can now use this as a recipe for your own maps. Or...

## Alternatives to Python

It's worth experimenting with alternatives if you intend to create more complex maps. For example, **Carto** is an option for producing choropleths. Here is a brief tutorial of how to map population density based on the csv we created (*borough_density.csv*) that has the unprojected borough polygons and their associated population densities. It's still sensible to do data linking and processing here in python, as it makes the final stages fairly straightforward - note that Carto works with latlon (unprojected) data rather than the projected/OSGB data.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("fofRRwZjiyg")